In [2]:
# import libraries
import pandas as p
import random 
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Read File
data = p.read_csv("influencers.csv", sep=",")
print(data)

     id  followerCount  engagementRate
0     1           6207             1.0
1     2          38237             5.8
2     3          14840             0.6
3     4          43851             1.4
4     5          15530             3.5
5     6          24050             3.3
6     7          44837             1.3
7     8          10029             3.4
8     9          14531             0.2
9    10           7862             2.8
10   11          95117             6.6
11   12          28548             2.5
12   13          90773             6.7
13   14          34711             1.8
14   15          11546             2.4
15   16          38046             0.9
16   17          17006             0.9
17   18          22159             1.0
18   19          55217             4.9
19   20           1908             2.3
20   21          23555             0.8
21   22           6463             1.2
22   23          27915             2.1
23   24           3628             0.5
24   25          77585   

In [4]:
# Determine the Input and Output
followCount = ["Low", "Average", "High"]
engRate     = ["Bottom", "Middle", "Upper"]
score       = ["Rejected", "Considered", "Accepted"]

In [5]:
d = data.values

In [6]:
# Design the Membership Function

# ------------------------------
# Follower Count (HIGH)
# ------------------------------
def follHigh(x):
    if ((x <= 55000) and (x > 4900)): return ((x-4900)/(55000-4900))
    elif (x <= 4900): return 0
    elif (x > 55000): return 1
# ------------------------------
# Follower Count (AVG)
# ------------------------------
def follAvg(x):
    if ((x <= 10000) or (x > 48600)): return 0
    elif ((x <= 20000) and (x > 10000)): return ((x-10000)/(20000-10000))
    elif ((x <= 45000) and (x > 20000)): return 1
    elif ((x > 45000) and (x <= 48600)): return ((48600 - x)/(48600-45000))
# ------------------------------
# Follower Count (LOW)
# ------------------------------
def follLow(x):
    if ((x <= 15000) and (x > 10000)): return ((15000-x)/(15000-10000))
    elif (x <= 10000): return 1
    elif (x > 15000): return 0      
    
# ------------------------------
# Engagement Rate (HIGH)
# ------------------------------
def engUpper(x):
    if ((x <= 8.0) and (x > 6.5)): return ((x-6.6)/(8.0-6.5))
    elif (x <= 6.5): return 0
    elif (x > 8.0): return 1
# ------------------------------
# Engagement Rate (MIDDLE)
# ------------------------------    
def engMid(x):
    if ((x <= 2.0) or (x > 7.5)): return 0
    elif ((x <= 4.5) and (x > 2.0)): return ((x-2.0)/(4.5-2.0))
    elif ((x <= 6.5) and (x > 4.5)): return 1
    elif ((x > 6.5) and (x <= 7.5)): return ((7.5 - x)/(7.5-6.5))
# ------------------------------
# Engagement Rate (LOW)
# ------------------------------
def engBottom(x):
    if ((x <= 4.5) and (x > 2.0)): return ((4.5-x)/(4.5-2.0))
    elif (x <= 2.0): return 1
    elif (x > 4.5): return 0


In [7]:
# Define the Rules

def rules(followCount, engRate):
    if ((followCount == "High") and (engRate == "Upper")):scoreValue = "Accepted"
    elif ((followCount == "High") and (engRate == "Middle")):scoreValue = "Accepted"
    elif ((followCount == "High") and (engRate == "Bottom")):scoreValue = "Considered" 
    #----------------------------------------------------------------------------------
    elif ((followCount == "Average") and (engRate == "Upper")):scoreValue = "Accepted"
    elif ((followCount == "Average") and (engRate == "Middle")):scoreValue = "Considered"
    elif ((followCount == "Average") and (engRate == "Bottom")):scoreValue = "Rejected"
    #----------------------------------------------------------------------------------
    elif ((followCount == "Low") and (engRate == "Upper")):scoreValue = "Accepted"
    elif ((followCount == "Low") and (engRate == "Middle")):scoreValue = "Rejected"
    elif ((followCount == "Low") and (engRate == "Bottom")):scoreValue = "Rejected"
    return scoreValue

In [8]:
def FuzziFollowers(value):
    #FUZZIFICATION for Followers Count
    high = follHigh(value)
    avg = follAvg(value)
    low = follLow(value)
    return high, avg, low

In [9]:
def FuzziEngagement(value):
    #FUZZIFICATION for ENGAGEMENT RATE
    upper = engUpper(value)
    middle = engMid(value)
    bottom = engBottom(value)
    
    return upper, middle, bottom

In [10]:
def InferenceTable(high, avg, low, upper, middle, bottom):
    # high, avg, low, upper, middle, bottom : nilainya
    tempAcc = []
    tempCons = []
    tempRej = []
    
    tempTable = [
        [high, upper], 
        [high, middle], 
        [high, bottom],
        [avg, upper], 
        [avg, middle], 
        [avg, bottom], 
        [low, upper], 
        [low, middle], 
        [low, bottom]
    ]
    
    tempNilai = 0
    
    for j in range(9):
        # Classification
        tempNilai = min(tempTable[j][0], tempTable[j][1])
        if (j == 0) or (j == 1) or (j == 3) or (j == 6):tempAcc.append(tempNilai)
        elif (j == 2) or (j == 4):tempCons.append(tempNilai)
        elif (j == 5) or (j == 7) or (j == 8):tempRej.append(tempNilai)
    
    acc = max(tempAcc)
    cons = max(tempCons)
    rej = max(tempRej)
    
    return acc, cons, rej

In [11]:
# Defuzzification using Takagi-Sugeno style

def Defuzz(accepted, considered, rejected):
    # constant variable
    rej = 60
    cons = 80
    acc = 100
    
    # equation 
    result = ((accepted*acc)+(considered*cons)+(rejected*rej))/(accepted + considered + rejected)
    return result

In [12]:
def writeFile(sortedData):
    # open and write the data
    f = open("result.txt", "a")
    d = str(sortedData)
    f.write(d + "\n")
    print()
    f.close()

In [31]:
#                                         MAIN PROGRAM                                               #
#                                        (FUZZY LOGIC)                                               #

person = []
tampungGan = []
personID = []

for i in range(len(data)):
    # FUZZIFICATION for FOLLOWERS COUNT
    high, avg, low = FuzziFollowers(d[i][1])
    #FUZZIFICATION for ENGAGEMENT RATE
    upper, middle, bottom = FuzziEngagement(d[i][2])
    # INFERENCE TABLE
    acc, cons, rej = InferenceTable(high, avg, low, upper, middle, bottom)
    # DEFUZZIFICATION
    res = Defuzz(acc, cons, rej)
    person.append({
        'id':i, 
        'hasil':res
    })
# SORTING 
srt = sorted(person, key=lambda x:x['hasil'],reverse=True)
print(srt)
for i in range(20):
    tampungGan.append(srt[i])
    writeFile(srt[i])

[{'id': 0, 'hasil': 60.50849106152859}, {'id': 1, 'hasil': 87.9909392715462}, {'id': 2, 'hasil': 65.81483778123574}, {'id': 3, 'hasil': 68.74802079707135}, {'id': 4, 'hasil': 76.77603369918442}, {'id': 5, 'hasil': 78.58541516245486}, {'id': 6, 'hasil': 68.87124182280618}, {'id': 7, 'hasil': 68.03205094743436}, {'id': 8, 'hasil': 65.95769240405043}, {'id': 9, 'hasil': 64.44388877775555}, {'id': 10, 'hasil': 99.99999999999999}, {'id': 11, 'hasil': 71.84791451971579}, {'id': 12, 'hasil': 100.0}, {'id': 13, 'hasil': 67.46105041859067}, {'id': 14, 'hasil': 68.5866237392486}, {'id': 15, 'hasil': 67.9633856281383}, {'id': 16, 'hasil': 65.12900250518683}, {'id': 17, 'hasil': 65.1244822518149}, {'id': 18, 'hasil': 100.0}, {'id': 19, 'hasil': 60.0}, {'id': 20, 'hasil': 65.42651443531379}, {'id': 21, 'hasil': 60.605075198885075}, {'id': 22, 'hasil': 67.391916843329}, {'id': 23, 'hasil': 60.0}, {'id': 24, 'hasil': 85.6}, {'id': 25, 'hasil': 66.67730355005983}, {'id': 26, 'hasil': 75.42418715804037